<a href="https://colab.research.google.com/github/smrutipunto/OPT/blob/main/opt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Practical 3: Simplex Algorithm and Duality
import numpy as np
from scipy.integrate import quad, dblquad

# --- Part 1: Basic Simplex Algorithm (Maximization) ---

def simplex_method(c, A, b):
    num_vars = len(c)
    num_constraints = len(b)

    tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))
    tableau[:num_constraints, :num_vars] = A
    tableau[:num_constraints, num_vars:num_vars + num_constraints] = np.eye(num_constraints)
    tableau[:num_constraints, -1] = b
    tableau[-1, :num_vars] = -c
    tableau[-1, -1] = 0

    print("Initial Tableau:\n", tableau)
    print("-" * 40)

    iteration = 0
    while np.any(tableau[-1, :-1] < 0):
        iteration += 1
        print(f"Iteration {iteration}:")

        pivot_col = np.argmin(tableau[-1, :-1])
        print(f"  Pivot Column (entering variable index): {pivot_col}")

        if np.all(tableau[:-1, pivot_col] <= 0):
            print("Problem is unbounded!")
            return None

        ratios = []
        for i in range(num_constraints):
            if tableau[i, pivot_col] > 0:
                ratios.append(tableau[i, -1] / tableau[i, pivot_col])
            else:
                ratios.append(np.inf)
        pivot_row = np.argmin(ratios)
        print(f"  Pivot Row (leaving variable index): {pivot_row}")

        pivot_element = tableau[pivot_row, pivot_col]
        print(f"  Pivot Element: {pivot_element}")

        tableau[pivot_row, :] = tableau[pivot_row, :] / pivot_element

        for i in range(num_constraints + 1):
            if i != pivot_row:
                factor = tableau[i, pivot_col]
                tableau[i, :] = tableau[i, :] - factor * tableau[pivot_row, :]

        print("  Tableau after pivot:\n", tableau)
        print("-" * 40)

    optimal_value = tableau[-1, -1]
    optimal_solution = np.zeros(num_vars)

    for j in range(num_vars):
        col = tableau[:, j]
        if np.count_nonzero(col) == 1 and np.all(col[col != 0] == 1):
            row_idx = np.where(col == 1)[0][0]
            if row_idx < num_constraints:
                optimal_solution[j] = tableau[row_idx, -1]

    return optimal_value, optimal_solution

if __name__ == "__main__":
    print("--- Simplex Method Example (Maximization) ---")

    c_simplex = np.array([3, 2])
    A_simplex = np.array([[1, 1],
                          [2, 1]])
    b_simplex = np.array([4, 6])

    optimal_val, optimal_sol = simplex_method(c_simplex, A_simplex, b_simplex)

    if optimal_val is not None:
        print("\nOptimal Value (Simplex):", optimal_val)
        print("Optimal Solution (Simplex - x1, x2):", optimal_sol[:len(c_simplex)])
    print("=" * 60)


--- Simplex Method Example (Maximization) ---
Initial Tableau:
 [[ 1.  1.  1.  0.  4.]
 [ 2.  1.  0.  1.  6.]
 [-3. -2.  0.  0.  0.]]
----------------------------------------
Iteration 1:
  Pivot Column (entering variable index): 0
  Pivot Row (leaving variable index): 1
  Pivot Element: 2.0
  Tableau after pivot:
 [[ 0.   0.5  1.  -0.5  1. ]
 [ 1.   0.5  0.   0.5  3. ]
 [ 0.  -0.5  0.   1.5  9. ]]
----------------------------------------
Iteration 2:
  Pivot Column (entering variable index): 1
  Pivot Row (leaving variable index): 0
  Pivot Element: 0.5
  Tableau after pivot:
 [[ 0.  1.  2. -1.  2.]
 [ 1.  0. -1.  1.  2.]
 [ 0.  0.  1.  1. 10.]]
----------------------------------------

Optimal Value (Simplex): 10.0
Optimal Solution (Simplex - x1, x2): [2. 2.]


In [ ]:
# --- Part 2: Duality ---

def form_dual_problem(c_primal, A_primal, b_primal):
    c_dual = b_primal
    A_dual = A_primal.T
    b_dual = c_primal
    return c_dual, A_dual, b_dual

if __name__ == "__main__":
    print("\n--- Duality Example ---")
    print("Primal Problem (from Simplex Example):")
    print("  Maximize Z = 3x1 + 2x2")
    print("  Subject to:")
    print("    x1 + x2 <= 4")
    print("    2x1 + x2 <= 6")
    print("    x1, x2 >= 0")

    c_primal_ex = np.array([3, 2])
    A_primal_ex = np.array([[1, 1],
                            [2, 1]])
    b_primal_ex = np.array([4, 6])

    c_dual_ex, A_dual_ex, b_dual_ex = form_dual_problem(c_primal_ex, A_primal_ex, b_primal_ex)

    print("\nCorresponding Dual Problem:")
    print(f"  Minimize W = {c_dual_ex[0]}y1 + {c_dual_ex[1]}y2")
    print("  Subject to:")
    print(f"    {A_dual_ex[0,0]}y1 + {A_dual_ex[0,1]}y2 >= {b_dual_ex[0]}")
    print(f"    {A_dual_ex[1,0]}y1 + {A_dual_ex[1,1]}y2 >= {b_dual_ex[1]}")
    print("    y1, y2 >= 0")

    print("\nRelationship between Primal and Dual:")
    print("The optimal value of the Primal problem (Maximization) will be equal to the optimal value of the Dual problem (Minimization).")
    try:
        print("From Simplex calculation, Primal Optimal Value (Z) =", optimal_val)
        print("Thus, the Dual Optimal Value (W) should also be =", optimal_val)
    except NameError:
        print("Primal Optimal Value not computed yet. Run Simplex first.")


--- Duality Example ---
Primal Problem (from Simplex Example):
  Maximize Z = 3x1 + 2x2
  Subject to:
    x1 + x2 <= 4
    2x1 + x2 <= 6
    x1, x2 >= 0

Corresponding Dual Problem:
  Minimize W = 4y1 + 6y2
  Subject to:
    1y1 + 2y2 >= 3
    1y1 + 1y2 >= 2
    y1, y2 >= 0

Relationship between Primal and Dual:
The optimal value of the Primal problem (Maximization) will be equal to the optimal value of the Dual problem (Minimization).
From Simplex calculation, Primal Optimal Value (Z) = 10.0
Thus, the Dual Optimal Value (W) should also be = 10.0
